In [17]:
import pandas as pd
import contractions
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from bs4 import BeautifulSoup
import re
import nltk
import tqdm
import unicodedata
import contractions
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import load_model

import pickle


In [5]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/adam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/adam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adam/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/adam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
newsLine = "So, Alphabet has an ROCE of 29%. That's a fantastic return and not only that, it outpaces the average of 6.4% earned by companies in a similar industry."

In [3]:
def strip_html_tags(text):
  """
  Removes html tags from text taken for html pages
  """
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  """
  Removes accents from chars
  """
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def stopwords_removal(words):
    """
    Remvoves stopwords (english language) from the text
    """
    list_stopwords = nltk.corpus.stopwords.words('english')
    return [word for word in words if word not in list_stopwords]

In [4]:
def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    #case folding
    doc = doc.lower()
    #remove special characters\whitespaces
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    #tokenize
    doc = word_tokenize(doc)
    #filtering
    doc = stopwords_removal(doc)
    norm_docs.append(doc)
  
  norm_docs = [" ".join(word) for word in norm_docs]
  return norm_docs

In [7]:
processedNewsLine =  pre_process_corpus([newsLine])

100%|██████████| 1/1 [00:00<00:00, 1214.33it/s]


In [13]:
with open('../cnn-models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [15]:
tokenizedNewsLine = tokenizer.texts_to_sequences(processedNewsLine)

In [19]:
MAX_LENGTH = 47
tokenizedNewsLine_padded = pad_sequences(tokenizedNewsLine, maxlen=MAX_LENGTH, padding="post")

In [20]:
tokenizedNewsLine_padded

array([[4058,  459, 7879,  682, 1133, 1244, 2511,   74, 1453,   90,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0]], dtype=int32)

In [21]:
model = load_model("../cnn-models/sentimentModel.keras")

In [29]:
list(pd.DataFrame(model.predict(tokenizedNewsLine_padded)).apply(lambda x:-1*x[0]+ 1*x[2],axis=1))

1/1 [==============================] - 0s 36ms/step


0    0.075209
dtype: float64